In [1]:
import pandas as pd
import numpy as np
import math
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
transactions = pd.read_csv('transactions.csv')

In [3]:
transactions.head()

,person,time,amount,offer_id,time_received,time_viewed,time_completed,reward,potential_reward,duration,...,email,social,web,age,gender,income,F,M,O,member_since_month
0,0009655768c64bdeb2e877511632db8f,168.0,NaN,5a8bc65990b245e5a138643cd4eb9837,168.0,192.0,NaN,0.0,0.0,72.0,...,1.0,1.0,0.0,33.0,M,72000.0,0.0,1.0,0.0,45.0
1,0009655768c64bdeb2e877511632db8f,228.0,22.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,33.0,M,72000.0,0.0,1.0,0.0,45.0
2,0009655768c64bdeb2e877511632db8f,336.0,NaN,3f207df678b143eea3cee63160fa8bed,336.0,372.0,NaN,0.0,0.0,96.0,...,1.0,0.0,1.0,33.0,M,72000.0,0.0,1.0,0.0,45.0
3,0009655768c64bdeb2e877511632db8f,414.0,8.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,33.0,M,72000.0,0.0,1.0,0.0,45.0
4,0009655768c64bdeb2e877511632db8f,528.0,14.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,33.0,M,72000.0,0.0,1.0,0.0,45.0


lets keep the rows which contain either transactions or offers that have actually been viewed - only those can have an impact onto the customer

In [4]:
transactions = transactions[(transactions.offer_type=='transaction') | (transactions.time_viewed.isnull()==False)].copy()
transactions.sort_values(['person','time','offer_id'],inplace=True)

define the helper columns to calculate a conversion rate

In [5]:
conv= transactions.copy()
conv.loc[: ,'converted'] = (conv.time_completed.isnull()==False)
conv.loc[:,'viewed'] = (conv.time_viewed.isnull()==False)

we define the over-all-conversion-rate as the ratio of viewed offers to converted meaning completed offers

In [6]:
conv_general = conv[conv['offer_id'].isnull()==False]['converted'].sum() / conv[conv['offer_id'].isnull()==False]['viewed'].sum()
conv_general

0

we can also split this conversion rate by calculating it on each offer type:  
conversion rate separated by offer types

In [7]:
def conversion_by_offer_type(offer_type, df):
    return df[(df['offer_type']==offer_type)]['converted'].sum() / df[(df['offer_type']==offer_type)]['viewed'].sum()

for offer in ['bogo','discount','informational']:
    print(offer,'- conversion: ',conversion_by_offer_type(offer,conv))


('bogo', '- conversion: ', 0)
('discount', '- conversion: ', 0)
('informational', '- conversion: ', 0)


we can already see that the conversion on an informational offer seems significantly less than offers that reward the customer

lets think about some features to be helpful in respect to prognosing how a customer might respond to an offer

- average time deltas between received and viewed
- average time deltas between viewed and completed
- columns for converted per offer_type
- number of transactions
- average spenditure
- accumulation of viewed offers
- accumulation of completed offers
- accumulated reward


In [8]:
df = conv.copy()

In [9]:
df.head(10).columns

Index([u'person', u'time', u'amount', u'offer_id', u'time_received',
       u'time_viewed', u'time_completed', u'reward', u'potential_reward',
       u'duration', u'offer_type', u'difficulty', u'mobile', u'email',
       u'social', u'web', u'age', u'gender', u'income', u'F', u'M', u'O',
       u'member_since_month', u'converted', u'viewed'],
      dtype='object')

The calculation of the mentioned features will be done on an iterative basis looping through the dataframe. a vector approach which is usually to be preferred looks much  more difficult to implement here. Columns that can be created on a vector approach will be directly calculated
First we need to initilize the values to be calculated

In [10]:
# indicating transaction
df['is_transaction'] = 0 
df.loc[df['offer_type']=='transaction', 'is_transaction'] = 1
# is an offer completed
df['is_completed'] = 0
df.loc[df['converted']==1, 'is_completed'] = 1

df['count_offers_completed'] = 0
df['count_offers_viewed'] = 0
df['count_transactions'] = 0
df['avg_spending'] = 0
df['avg_reward'] = 0
df['delta_time_reception_viewed'] = df['time_viewed'] - df['time_received']
df['delta_time_viewed_completion'] = df['time_completed'] - df['time_viewed']
df['delta_time_reception_viewed_avg'] = 0
df['delta_time_viewed_completion_avg'] = 0

In [ ]:
# the following computations are only allowed, if the iteration is within the same "customer domain". when customers
# change from one row to the next the calculated values may not be carried over otherwise resulting in false numbers
df['prev_person'] = df['person'].shift(1)
df.reset_index(drop=True,inplace=True)
for i, row in df.iterrows():
    if i % 10000 == 0: # processing indication
        print(i)
    if row['person'] == row['prev_person']: # computation only valid if within same person domain
        df.loc[i, 'count_offers_completed'] = df.loc[i-1, 'count_offers_completed'] + df.loc[i-1, 'is_completed'] #count completed offers
        df.loc[i, 'count_offers_viewed'] = df.loc[i-1, 'count_offers_viewed'] + (1 if df.loc[i-1, 'offer_type'] != 'transaction' else 0) #add viewed offer count
        df.loc[i, 'count_transactions'] = df.loc[i-1, 'count_transactions'] + df.loc[i-1, 'is_transaction'] # count transactions done
        df.loc[i, 'delta_time_reception_viewed_avg'] = np.nansum((df.loc[i-1, 'delta_time_reception_viewed_avg'], df.loc[i-1, 'delta_time_reception_viewed'])) #add time delta for later averaging
        df.loc[i, 'delta_time_viewed_completion_avg'] = np.nansum((df.loc[i-1, 'delta_time_viewed_completion_avg'], df.loc[i-1, 'delta_time_reception_viewed_avg']))#add time delta for later averaging

        df.loc[i, 'avg_spending'] = df.loc[i-1, 'avg_spending'] + (0 if df.loc[i-1, 'is_transaction'] == 0 else df.loc[i-1, 'amount']) #accumulate spending
        df.loc[i, 'avg_reward'] = np.nansum((df.loc[i-1, 'avg_reward'], df.loc[i-1, 'reward'])) #accumulate rewards

# Calculate the averages from the sums
df['delta_time_reception_viewed_avg'] /= df['count_offers_viewed']
df['delta_time_viewed_completion_avg'] /= df['count_offers_completed']
df['avg_spending'] /= df['count_transactions']
df['avg_reward'] /= df['count_offers_completed']

# treat nan
df['delta_time_reception_viewed_avg'].fillna(0, inplace=True)
df['delta_time_viewed_completion_avg'].fillna(0, inplace=True)
df['avg_spending'].fillna(0, inplace=True)
df['avg_reward'].fillna(0,inplace=True)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [ ]:
for offer_type in ['bogo','discount','informational']:
    df.loc[(df['offer_type']==offer_type)&(df['time_completed'].isnull()==False),offer_type] = 1
    df.loc[(df['offer_type']==offer_type)&(df['time_completed'].isnull()==True),offer_type] = 0

In [ ]:
df[df['offer_type']=='discount']

lets try to get an overview on categorical and numerical features by creating some data frames of conversion

In [ ]:
def sturges_binning(feature, df, correction=1):
    '''
    function that helps to make the right amount of bins
    '''
    binning_series = df[feature]
    min_ = min(binning_series)
    max_ = max(binning_series)
    len_ = len(binning_series)
    sturges_bin_width = np.ceil((max_ - min_)/(np.ceil(math.log(len_, 2))+correction))
    return np.arange(min_,max_+sturges_bin_width,sturges_bin_width)

In [ ]:
def feature_df(feature,df,numerical=True, correction=1):
    '''
    create data frames with a groupby on calculated bins regarding the feature given
    '''
    narrow = [feature]+['bogo','discount','informational']
    columns = ['bogo','discount','informational']
    if numerical:
        results = df[narrow].groupby(pd.cut(df[feature],bins=sturges_binning(feature,df,correction))).mean().drop(feature,axis=1)
    else:
        results =  df[narrow].groupby([feature]).mean()
    return results

In [ ]:
df.columns

In [ ]:
### todo put it in data folder
data_dir = './data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
feature_path = os.path.join(data_dir,'features.csv')
if os.path.exists(feature_path):
    df = pd.read_csv(feature_path)
else:
    df.to_csv(feature_path)

In [ ]:
import seaborn as sns


lets display the conversion rates based on different features such as **gender**, **income**, **member_since_month**, **avg_spending** and **count_transactions**

In [ ]:
display(feature_df('gender',df,False))
display(feature_df('income',df,numerical=True,correction=-5))
display(feature_df('member_since_month',df,numerical=True))
display(feature_df('avg_spending',df,numerical=True))
display(feature_df('count_transactions',df,numerical=True))

we can see in the data frame overview on top that high spendings do not give us any benefit for modelling regarding conversion rate since most of the time there is no data

In [ ]:
ax = sns.heatmap(feature_df('gender',df,False))
ax.set_title('distribution of conversion')

male customers seem to complete discount offers more likely than bogo offers  
femalecustomers tend to complete both types of offers more than male customers. again discount seems more attractive than bogo

In [ ]:
display(sns.heatmap(feature_df('income',df,numerical=True,correction=-5)))
ax.set_title('distribution of conversion by income')

In [ ]:
display(sns.heatmap(feature_df('member_since_month',df,numerical=True)))

In [ ]:
display(sns.heatmap(feature_df('avg_spending',df,numerical=True)))

In [ ]:
ax = (sns.heatmap(feature_df('count_transactions',df,numerical=True)))
ax.set_title('distribution of conversion')

we must determine which columns look interesting for a model to train. from the above mentioned features we take the following

In [ ]:
df.columns

In [ ]:
remove_features = ['person','time','amount', 'offer_id', 'time_received', 'time_viewed', 'mobile', 'email', 'social', 'web','converted','delta_time_reception_viewed','prev_person',
       'delta_time_viewed_completion','gender','O','time_completed', 'reward', 'potential_reward', 'duration','offer_type', 'difficulty','is_transaction', ]
kept_features = [ 'person',
       'age', 'income', 'F', 'M', 'member_since_month','viewed',  'is_completed',
       'count_offers_completed', 'count_offers_viewed', 'count_transactions','avg_spending', 'avg_reward',  'delta_time_reception_viewed_avg',
       'delta_time_viewed_completion_avg', 'bogo','discount', 'informational']

In [ ]:
df[kept_features]

In [ ]:
bogo = df[df.bogo == 1]
bogo_kept= bogo[kept_features]
bogo_kept.info()